In [1]:
from astropy.time import Time, TimeDelta
from datetime import datetime
import pandas as pd
import numpy as np
from astropy import units as u
from matplotlib import pyplot as plt

import lsst.daf.butler as dafButler

from lsst.summit.utils import BestEffortIsr, makeDefaultLatissButler
from lsst.pipe.tasks.quickFrameMeasurement import QuickFrameMeasurementTask
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfConfig
from lsst.pex.exceptions import InvalidParameterError
from lsst.ts.observatory.control.constants import latiss_constants
from lsst.geom import PointD
# below is an unused import, but is required to use the `psfex` fitter
import lsst.meas.extensions.psfex.psfexPsfDeterminer # noqa: F401

from lsst_efd_client import EfdClient

%matplotlib inline

In [2]:
REPO_DIR = '/repo/LATISS'

#butler = dafButler.Butler(REPO_DIR, instrument='LATISS', collections = 'LATISS/raw/all')
butler = makeDefaultLatissButler()
# Instantiate bestEffort 
best_effort_isr = BestEffortIsr()

### Declare QuickFrameMeasurement tasks
These are run if the measurePsf method fails. It provides a reasonable yet less-accurate representation.

In [3]:
qm_config = QuickFrameMeasurementTask.ConfigClass()
qm = QuickFrameMeasurementTask(config=qm_config)

Test image

In [6]:
registry = butler.registry

In [ ]:
dataset_refs

Define 

In [4]:
client = EfdClient('summit_efd')

In [6]:
dataId = {'detector': 0, 'day_obs': 20220929, 'seq_num': 126}

In [7]:
best_effort_isr.doWrite = False  # Don't write to butler database
exp = best_effort_isr.getExposure(dataId, skipCosmics=False)

In [8]:
def measurePsf(exp):
    platescale = latiss_constants.pixel_scale

    imCharConfig = CharacterizeImageTask.ConfigClass()
    imCharConfig.doMeasurePsf = True
    imCharConfig.useSimplePsf = True
    
    imCharConfig.doApCorr = False
    imCharConfig.doDeblend = False
    
    installConfig = InstallGaussianPsfConfig()
    exp.setPsf(None)  # if not set to none, fwhm max para is ignored
    installConfig.fwhm = 15
    installConfig.width = 61
    
    imCharConfig.installSimplePsf = installConfig    
    
    imCharConfig.detection.includeThresholdMultiplier = 5
    imCharConfig.measurePsf.starSelector['objectSize'].doFluxLimit = True
    imCharConfig.measurePsf.starSelector['objectSize'].fluxMin = 12500.0
    imCharConfig.measurePsf.starSelector['objectSize'].fluxMax = 0.0
    imCharConfig.measurePsf.starSelector['objectSize'].doSignalToNoiseLimit = False
    imCharConfig.measurePsf.starSelector['objectSize'].signalToNoiseMin = 20.0
    imCharConfig.measurePsf.starSelector['objectSize'].signalToNoiseMax = 0.0
    imCharConfig.measurePsf.starSelector['objectSize'].widthMin = 0.0
    imCharConfig.measurePsf.starSelector['objectSize'].widthMax = 80.0  # default 10
    imCharConfig.measurePsf.starSelector['objectSize'].sourceFluxField = "base_GaussianFlux_instFlux"
    imCharConfig.measurePsf.starSelector['objectSize'].widthStdAllowed = 0.15 # 0.15 default
    imCharConfig.measurePsf.starSelector['objectSize'].nSigmaClip = 2.0
    
    imCharConfig.measurePsf.psfDeterminer.name = 'psfex'
    imCharConfig.measurePsf.psfDeterminer['psfex'].spatialOrder = 1
    imCharConfig.psfIterations=1
    
    imCharConfig.background.binSize = 2000
    imCharConfig.background.approxOrderX = 2

    imCharConfig.detection.background = imCharConfig.background
    
    imCharTask = CharacterizeImageTask(config=imCharConfig)

    result = imCharTask.run(exp)

    psf = exp.getPsf()
    ixx = psf.computeShape(exp.getBBox().getCenter()).getIxx()
    iyy = psf.computeShape(exp.getBBox().getCenter()).getIyy()
    psfShape = psf.computeShape(exp.getBBox().getCenter()).getDeterminantRadius()
    
    fwhmX = np.sqrt(ixx)*2.355*platescale
    fwhmY = np.sqrt(iyy)*2.355*platescale
    
    overallFwhm = psfShape * 2.355 * platescale
    print(f"Psf shape from imChar task (x,y) = ({fwhmX:.3f}, {fwhmY:.3f}) FWHM arcsec")
    return fwhmX, fwhmY, overallFwhm, psf

In [9]:
def get_FWHM(exp):
    brightest_source_centroid = []
    fwhmX=None; fwhmY=None; overallFwhm=None
    result=None
    psf=None
    success=False
    try:
        fwhmX, fwhmY, overallFwhm, psf = measurePsf(exp)
        success=True
        pass
    except InvalidParameterError as e:
        print('Caught the InvalidParameterError, measurePsf was not successful')
        #print(f'error is {e}')
        pass
    except RuntimeError as e:
        print('Caught the RuntimeError, measurePsf was not successful')
        #print(f'error is {e}')
        pass

    if not success:
        print('Using Merlin\'s simplified algorithm')
        exp = best_effort_isr.getExposure(dataId, skipCosmics=False)
        result = qm.run(exp)
        brightest_source_centroid.append(result)
        if result.success:
            fwhmX=result.medianXxYy[0]
            fwhmY=result.medianXxYy[1]
            overallFwhm=np.sqrt(result.medianXxYy[0]**2 + result.medianXxYy[1]**2)
            print(f"Psf shape from QuickFrameMeasurementTask task (x,y) = ({fwhmX:.3f}, {fwhmY:.3f}) FWHM arcsec")
        else:
            raise RuntimeError('No PSF could be derived using either method')
            
    return fwhmX, fwhmY, overallFwhm, corr_dimm_seeing

In [22]:
def corr_dimm_seeing(exp):
    filter_band=exp.getInfo().getFilterLabel().bandLabel 
    airmass=exp.getInfo().getMetadata().get('AMSTART')#+exp.getInfo().getMetadata().get('AMEND'))/2.0)
    elevation=exp.getInfo().getMetadata().get('ELSTART')#+exp.getInfo().getMetadata().get('ELEND'))/2.0)
    dimm_seeing=exp.getInfo().getMetadata().get('SEEING')
    
    airmass_corr_dimm_seing = dimm_seeing * (airmass ** -0.6)
    
    filter_wavelength= {'white': 700, 'g': 475, 'r': 623, 'i': 763, 'z': 828.0}
    
    corr_dimm_seeing = airmass_corr_dimm_seeing * ((500. / filter_wavelength[filter_band]) ** -0.2)


In [ ]:
dimm_seeing = 1.8
airmass = 1.3

filter_wavelength= {'white': 700, 'g': 475, 'r': 623, 'i': 763, 'z': 828.0}
filter_band='g'

airmass_corr_dimm_seeing = dimm_seeing * (airmass ** 0.6)
print(airmass_corr_dimm_seing)
corr_dimm_seeing = airmass_corr_dimm_seeing * ((500. / filter_wavelength[filter_band]) ** -0.2)


In [23]:
corr_dimm_seeing(exp)

/tmp/ipykernel_20782/1950371799.py:2: FutureWarning: Call to deprecated method getFilterLabel. (Replaced by getFilter. Will be removed after v24.) -- Deprecated since version v24.0.
  filter_band=exp.getInfo().getFilterLabel().bandLabel


ZeroDivisionError: 0.0 cannot be raised to a negative power

#### Derive Corrections for elevation (airmass) and wavelength

Capture metadata for calculation and comparison

In [ ]:
filter_band=exp.getInfo().getFilterLabel().bandLabel 
airmass=((exp.getInfo().getMetadata().get('AMSTART')+exp.getInfo().getMetadata().get('AMEND'))/2.0)
elevation=((exp.getInfo().getMetadata().get('ELSTART')+exp.getInfo().getMetadata().get('ELEND'))/2.0)
dimm_seeing=exp.getInfo().getMetadata().get('SEEING')

In [ ]:
print(f"Target at elevation = {elevation:0.1f}, airmass = {airmass:0.3f}")
print(f"PSF FWHM (x,y) = ({fwhmX:.3f}, {fwhmY:.3f}) FWHM [arcsec]")
print(f"Overall PSF FWHM is = {overallFwhm:.3f} FWHM [arcsec]")

Calculate airmass corrected FWHM

In [ ]:
airmass_corr_fwhm = overallFwhm * (airmass ** -0.6)

Filter wavelengths are not currently in the fits headers.
Must use an ugly dictionary until this is implemented.

In [ ]:
filter_wavelength= {'white': 700, 'g': 475, 'r': 623, 'i': 763, 'z': 828.0}

#### Derive airmass+wavelength corrected FWHM

In [ ]:
corr_fwhm = airmass_corr_fwhm * ((500. / filter_wavelength[filter_band]) ** -0.2)
print(f'Airmass and wavelength corrected FWHM: {corr_fwhm:0.3f} [arcsec]')
print(f'DIMM reported FWHM: {dimm_seeing:0.3f} [arcsec]')

*** Define here the sequences and the observation day dayObs

In [ ]:
dayObs = '2021-10-05'
expList = range(456, 507+1)
print(expList)

Loop through the data [456 - 507+1]

In [ ]:
#Initialize the lists
exposiciones, comienzo, fin, azimuth, elevation, masaAire, filtros, tiempoExposicion, seeing, fwhm, fwhmX, fwhmY = [], [], [], [], [], [], [], [], [], [], [], []

for exposure in expList:
    print(f'The sequence is', exposure)
    
    expId = int(str(dayObs[0:4])+str(dayObs[5:7])+str(dayObs[8:10])+'00'+str(exposure))
    
    exposiciones.append(expId)
    
    data_id = {'instrument': 'LATISS', 'exposure': expId , 'detector':0}

    postIsr = bestEffort.getExposure(data_id)   

    mData = postIsr.getMetadata()        
    
    comienzo.append(mData['DATE-BEG'])
    fin.append(mData['DATE-END'])
    
    azimuth.append(mData['AZSTART'])
    elevation.append(mData['ELSTART'])
    
    masaAire.append(mData['AMSTART'])
    seeing.append(mData['SEEING'])
    
    filtros.append(mData['FILTER'])
    tiempoExposicion.append(mData['EXPTIME'])
    
    # Compute PSF/FWHM
    try:
        fwhm_in_x, fwhm_in_y, overallFwhm = measurePsf(postIsr)        
        fwhm.append(overallFwhm)
        fwhmX.append(fwhm_in_x)
        fwmhY.append(fwhm_in_y)
        
    except:
        print('No hay datos suficientes para obtener PSF')
        fwhm.append('PSF computation not possible')
print(fwhm)   
    

In [ ]:
# Build Panda Frame and save it to a file
#import pickle
exposure_frame = pd.DataFrame(list(zip(exposiciones, comienzo, fin, azimuth, elevation, masaAire, filtros, tiempoExposicion, seeing, fwhm, fwmhX, fwmhY)),columns =['expId', 'timeStart', 'timeEnd', 'AzStart', 'ElStart', 'Airmass', 'Filter', 'ExpTime', 'Seeing', 'FWHMImage', 'FWMH_X', 'FWHM_Y'])       
exposure_frame.to_pickle('upanddown_felh0600_ellipticity.txt')

***
Read Panda Frame 

In [ ]:
exposure_frame = pd.read_pickle('upanddown_felh0600_ellipticity.txt')
exposure_frame
#plt.plot(exposure_frame.Airmass, exposure_fram)

In [ ]:
the_ones_with_PSF = exposure_frame.loc[exposure_frame['FWHMImage'] != 'PSF computation not possible']
the_ones_with_PSF

Now we plot: 

In [ ]:
#the_ones_with_PSF.plot.scatter(x ='Airmass', y='FWHMImage')
plt.plot(the_ones_with_PSF.Airmass, the_ones_with_PSF.FWHMImage, linestyle = '', marker = 'o')
plt.plot(the_ones_with_PSF.Airmass, the_ones_with_PSF.Seeing, linestyle = '', marker = '*')

plt.legend(['FWHM Image', 'DIMM Seeing'])
plt.xlabel('Airmass')
plt.ylabel('FWHM (arcsec)')
plt.title('FWHM (arcsec) vs Airmass', fontsize = 14)
plt.show()
plt.savefig('FWHM_Image and DIMM Seeing vs Airmass')

# DIMM seeing is to 1.0 airmass. Correct DIMM Seeing to the observed airmass. 
plt.plot(the_ones_with_PSF.Airmass, the_ones_with_PSF.FWHMImage, linestyle = '', marker = 'o')
plt.plot(the_ones_with_PSF.Airmass, the_ones_with_PSF.Seeing*the_ones_with_PSF.Airmass**0.6, linestyle = '', marker = '*')
plt.legend(['FWHM Image', 'DIMM Airmass-corrected Seeing'])
plt.xlabel('Airmass')
plt.ylabel('FWHM (arcsec)')
plt.title('FWHM (arcsec) vs Airmass', fontsize = 14)
plt.show()
plt.savefig('FWHM_Image and Airmass-Corrected DIMM Seeing vs Airmass')


In [ ]:
# DIMM seeing is to 1.0 airmass. Correct DIMM Seeing to the observed airmass. 
%matplotlib inline
plt.plot(the_ones_with_PSF.Seeing*the_ones_with_PSF.Airmass**0.6, the_ones_with_PSF.FWHMImage, linestyle = '', marker = 'o')
plt.xlabel('DIMM Airmass-corrected Seeing')
plt.ylabel('FWHM Image (arcsec)')
plt.title('Image FWHM (arcsec) vs DIMM Airmass-corrected Seeing', fontsize = 14)
plt.show()
plt.savefig('FWHM_Image vs DIMM Airmass-Corrected Seeing')


In [ ]:
# El FWHM de la imagen frente al azimuth
plt.plot(exposure_frame.Airmass, linestyle = '', marker = '*')
#plt.plot(elevation, fwhm_dimm, linestyle = '', marker = 'o')
plt.xlabel('Azimuth (deg)')
plt.ylabel('FWHM Image (arcsec)')
plt.title('20210817 \n Computed image FWHM vs. Azimuth (deg)', fontsize = 14)
#print(fwhm_dimm)
plt.show()

In [ ]:
import matplotlib.dates as mdates

# Versus time
fig, ax = plt.subplots(num="FWHM Image & FWHM measured by DIMM", dpi=120)

# Plot Data
#if exposure_frame.FWHM_image.item != 'PSF computation not possible':
plt.plot(the_ones_with_PSF.timeStart, the_ones_with_PSF.Airmass)
plt.plot(the_ones_with_PSF.timeStart, the_ones_with_PSF.FWHMImage)
plt.plot(the_ones_with_PSF.timeStart, the_ones_with_PSF.Seeing)
plt.plot(the_ones_with_PSF.timeStart, the_ones_with_PSF.Seeing*the_ones_with_PSF.Airmass**0.6)

plt.legend(['Airmass', 'FWHMImage', 'Seeing', 'Airmass-Corrected Seeing'])
plt.xticks(range(1,50,5),rotation=45)
plt.show()
#ax.plot(exposure_frame.timeStart,exposure_frame.FWHM_image)


# Display title and sub-title
#ax.set_title(f"Weather Station\nWind Speed on {df.index[0]:%Y-%m-%d}")

# Setting grid
ax.grid(lw='0.5', alpha=0.2, ls='-')

# Rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them.
#fig.autofmt_xdate()

# Display Plot


In [ ]:
#  FWHM frente a elevacion
plt.plot(the_ones_with_PSF.ElStart, the_ones_with_PSF.FWHMImage, linestyle = '', marker = '*')
plt.plot(the_ones_with_PSF.ElStart, the_ones_with_PSF.Seeing*the_ones_with_PSF.Airmass**0.6, linestyle = '', marker = 'o')
plt.legend(['FWHM Image', 'DIMM Airmass-corrected Seeing'])
plt.xlabel('Elevation (deg)')
plt.ylabel('FWHM Image (arcsec)')
plt.title('Computed image FWHM and seeing vs. Elevation (deg)', fontsize = 14)
#plt.show()

In [ ]:
# Both FWHM vs time 
plt.plot(comienzo, fwhm_image, linestyle = '--', marker = '*')
plt.plot(comienzo, fwhm_dimm, linestyle = '--', marker = 'o')
plt.legend(['FWHM Image', 'DIMM Airmass-corrected Seeing'])
plt.xticks(range(1,len(comienzo),10),rotation=45)
plt.xlabel('Time')
plt.ylabel('FWHM (arcsec)')
plt.title('20210817 \n Image and DIMM FWHM vs. Time', fontsize = 14)


Now, let's try to find more DIMM data expanding the interval +-15 s

In [ ]:
from lsst_efd_client import EfdClient

exposiciones2 = []
comienzo2 = []
fin2 = []
azimuth2 = []
elevation2 = []
fwhm_dimm2 = []
fwhm_image2 = []

for i, exposure in enumerate(exposure_15s): 
    print(f'15 second exposure is', i,exposure)
    expId = exposure[0]
    print(expId)
    exposiciones2.append(expId)

    mData = butler.get('raw', dataId={'detector':0, 'expId':expId}).getMetadata()
    
    tStart = mData['DATE-BEG']
    tEnd = mData['DATE-END']
    
    comienzo2.append(mData['DATE-BEG'])
    fin2.append(mData['DATE-END'])
    
    azimuth2.append(mData['AZSTART'])
    elevation2.append(mData['ELSTART'])
    
    
#     #get dimm data
    client = EfdClient('summit_efd')
    t_start = Time(tStart, scale='tai')
    t_start2 = t_start - TimeDelta(25, format='sec')
    print(t_start2)
    t_end = Time(tEnd, scale='tai')
    t_end2 = t_end + TimeDelta(25, format='sec')
    print(t_end2)
    try:
        dimm = await client.select_time_series("lsst.sal.DIMM.logevent_dimmMeasurement", ['*'], t_start2, t_end2)
        fwhmDi = dimm.fwhm.tolist()[0]
        fwhm_dimm2.append(fwhmDi)  
        print(fwhmDi)
    except:
        print(f'No hay valores del DIMM')
        fwhm_dimm2.append(0)



# #     #Measure PSF and FWHM from image - continue the for loop
#     # First ISR task
    raw = butler.get('raw', expId=expId)
    bias = butler.get('bias', expId=expId)
    defects = butler.get('defects', expId=expId)
    postIsr = isrTask.run(raw, bias=bias, defects=defects).exposure
    plt.figure(figsize=(5,5))
    data = postIsr.image.array
    plt.imshow(data, vmin=np.percentile(data, 2),vmax=np.percentile(data, 98))
    plt.show()
    
# #   # Characterize image task and compute PSF/FWHM
    imCharTask = CharacterizeImageTask()
    try:
        imCharTask.run(postIsr)
        psf = postIsr.getPsf()
        shape = psf.computeShape()
        fwhmIm = shape.getDeterminantRadius()*2.355*0.097
        fwhm_image2.append(fwhmIm)
    except:
        print('No hay datos suficientes para obtener PSF')
        fwhm_image2.append(0)
print(fwhm_image2)   

In [ ]:
print(len(fwhm_dimm2))

In [ ]:
# #print(fwhm_image2)
# print(len(fwhm_image2))
# fwhm_image3 = fwhm_image2
# for value in range(0,len(fwhm_image3)):
#     if fwhm_image3[value] == 0:
#         del fwhm_image3[value]
#         #print(fwhm_image3)
# median_image3 = np.median(fwhm_image3)
# print(len(fwhm_image2))
# print(len(fwhm_image3))

# print(len(fwhm_dimm2))
# fwhm_dimm3 = fwhm_dimm2
# for value in range(0,len(fwhm_dimm3)):
#     if fwhm_dimm3[value] == 0:
#         del fwhm_dimm3[value]
#         #print(fwhm_dimm3)
# median_dimm3 = np.median(fwhm_dimm3)
# print(len(fwhm_image2))
# print(len(fwhm_image3))

# print(median_image3, median_dimm3)

In [ ]:
# Both FWHM vs time 
plt.plot(comienzo2, fwhm_image2, linestyle = '', marker = '*')
plt.plot(comienzo2, fwhm_dimm2, linestyle = '', marker = 'o')
plt.legend(['PSF FWHM', 'DIMM FWHM'])
plt.xticks(range(1,len(comienzo),10),rotation=45)
plt.xlabel('Time')
plt.ylabel('FWHM (arcsec)')
plt.title('20210817 \n PSF & DIMM FWHM Image vs. Time', fontsize = 14)
plt.show()


#Plots (when extended search for DIMM measurements +-20 seconds around image exposure)
# Plot the FWHM 
print(len(fwhm_image))
print(len(fwhm_dimm))
plt.plot(fwhm_dimm2, fwhm_image2, linestyle = '', marker = 'x')
plt.xlabel('DIMM FWHM (arcsec)')
plt.ylabel('PSF FWHM (arcsec)')
plt.title('20210817 15-second exposures \n PSF FWHM vs. DIMM FWHM', fontsize = 14)
#print(fwhm_dimm)
plt.show()

# FWHM frente al azimuth
plt.plot(azimuth2, fwhm_image2, linestyle = '', marker = '*')
plt.plot(elevation2, fwhm_dimm2, linestyle = '', marker = 'o')
plt.xlabel('Azimuth (deg)')
plt.ylabel('FWHM (arcsec)')
plt.legend(['PSF FWHM', 'DIMM FWHM'])
plt.title('20210817 \n PSF FWHM vs. Azimuth', fontsize = 14)
plt.show()

# FWHM  frente a elevacion
plt.plot(elevation2, fwhm_image2, linestyle = '', marker = '*')
plt.plot(elevation2, fwhm_dimm2, linestyle = '', marker = 'o')
plt.xlabel('Elevation (deg)')
plt.ylabel('FWHM (arcsec)')
plt.legend(['PSF FWHM', 'DIMM FWHM'])
plt.title('20210817 \n PSF FWHM vs. Elevation', fontsize = 14)
#print(fwhm_dimm)
plt.show()

plt.savefig('/home/isotuela/notebooks/Analysis_Notebooks/20210817_FWHM/' + 'fwhm')

Now with an interval of +- 25. Does it change that much? How often does the DIMM publish a new value? 

In [ ]:
exp = butler.get('raw', detector =0, expId= expId)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(10,10))
data = exp.image.array
plt.imshow(data, vmin=np.percentile(data, 2),vmax=np.percentile(data, 98))

In [ ]:
del(exp)

In [ ]:
from lsst.ip.isr.isrTask import IsrTask
isrConfig = IsrTask.ConfigClass()
isrConfig.doLinearize = False
isrConfig.doBias = True
isrConfig.doFlat = False
isrConfig.doDark = False
isrConfig.doFringe = False
isrConfig.doDefect = False
isrConfig.doWrite = False

isrTask = IsrTask(config=isrConfig)

In [ ]:
raw = butler.get('raw', expId=expId)
bias = butler.get('bias', expId=expId)
defects = butler.get('defects', expId=expId)

In [ ]:
postIsr = isrTask.run(raw, bias=bias, defects=defects).exposure

In [ ]:
plt.figure(figsize=(10,10))
data = postIsr.image.array
plt.imshow(data, vmin=np.percentile(data, 2),vmax=np.percentile(data, 98))

In [ ]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('firefly')
display1 = afwDisplay.Display(frame=1)

In [ ]:
display1.mtv(postIsr)

In [ ]:
from lsst.rapid.analysis import ImageExaminer
#from lsst.imageExaminer import ImageExaminer
#exp.getMaskedImage().getArrays()

In [ ]:
imExam = ImageExaminer(postIsr)

In [ ]:
%matplotlib inline

In [ ]:
imExam.plot()

In [ ]:
imExam = ImageExaminer(postIsr[0:2000, 0:2000])
imExam.plot()

In [ ]:
postIsr.mask.getMaskPlaneDict()

In [ ]:
postIsr.mask[2000,2000]

In [ ]:
postIsr.mask.interpret(postIsr.mask[1848,2307])

In [ ]:
md = postIsr.getMetadata()

In [ ]:
md.toDict()

In [ ]:
info = postIsr.getInfo()

In [ ]:
from astro_metadata_translator import ObservationInfo

In [ ]:
obsInfo = ObservationInfo(postIsr.getMetadata())

In [ ]:
obsInfo.boresight_airmass

In [ ]:
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask

In [ ]:
imCharTask = CharacterizeImageTask()

In [ ]:
imCharTask.run(postIsr)

In [ ]:
psf = postIsr.getPsf()

In [ ]:
kernelImage = psf.computeKernelImage()

In [ ]:
plt.imshow(kernelImage.array)

In [ ]:
psf.computeShape()

In [ ]:
shape = psf.computeShape()

In [ ]:
shape.getDeterminantRadius()*2.355*0.097

In [ ]:
from lsst.geom import Point2D

In [ ]:
psf.computeShape(position=Point2D(123.4,234.5))

In [ ]:
expId = 2021081700142
mData = butler.get('raw', dataId={'detector':0, 'expId':expId}).getMetadata()
    
t1 = mData['DATE-BEG']

expId2 = 2021081700541
mData = butler.get('raw', dataId={'detector':0, 'expId':expId2}).getMetadata()

    
t2 = mData['DATE-END']
    
    
#     #get dimm data
client = EfdClient('summit_efd')
t_t1 = Time(t1, scale='tai')
t_t2 = Time(t2, scale='tai')
alldimm = await client.select_time_series("lsst.sal.DIMM.logevent_dimmMeasurement", ['*'], t_t1, t_t2)
        

In [ ]:
alldimm.columns

In [ ]:
alldimm.fwhm